In [1]:
import numpy as np
import sys

In [2]:
# Function to calculate size of object in Python in MB
def sizeof(obj):
    objsize = sys.getsizeof(obj) / 1024 / 1024
    print(f"Size: {np.round(objsize,3)} MB")
    return objsize

# Step 1: Generate Random Embeddings


In [3]:
# Generate random embeddings
def generate_random_embeddings(num_embeddings, dim):
    embeddings = np.random.rand(num_embeddings, dim).astype(np.float32)
    return embeddings

# Add numpy seed for reproducibility
np.random.seed(32)

# Example: Create 1000 random embeddings with 128 dimensions
num_embeddings = 1000
dim = 128
embeddings = generate_random_embeddings(num_embeddings, dim)
_ = sizeof(embeddings)


Size: 0.488 MB


# Step 2: HNSW Node and HNSW Class with Corrected Level Selection

## 2.1 Define HNSW Node Class


In [42]:
class HNSWNode:
    def __init__(self, embedding, max_level, label=None):
        self.embedding = embedding
        self.label = label
        # Initialize neighbors for all levels, even those higher than the node's assigned level
        self.neighbors = {i: [] for i in range(max_level + 1)}

    def add_neighbor(self, neighbor, level):
        self.neighbors[level].append(neighbor)


## 2.2 Define HNSW Class with Corrected Level Selection

Step-by-Step Guide: Adding a New Node

**1. Initialize Level for the New Node (_get_random_level):**

The algorithm determines the level at which the new node will be placed. This level is chosen using a probabilistic approach, where higher levels are less likely to be chosen. The node will also be added to all levels below the selected level.

2. Create the New Node (add_node):

A new HNSWNode instance is created with the generated embedding and the determined level.

3. Check for the First Node (add_node):

If the HNSW structure is empty (i.e., no entry point exists), this new node becomes the entry_point, which is the starting node for all future searches. The process ends here for the first node.

4. Insert Node into the Graph (_insert_node):

If the graph already has an entry point, the algorithm begins the process of inserting the new node into the existing structure. This involves finding the best position for the new node by navigating through the graph from the top level down to the level of the new node.

5. Search for the Best Insertion Point (_search_layer):

Starting from the top level of the graph and the entry_point, the algorithm searches for the closest existing node to the new node’s embedding. It does this by comparing distances between the new node’s embedding and the embeddings of the neighbors at each level. The search continues until it finds the closest node or cannot improve the proximity by moving to another neighbor.

6. Connect the New Node to the Nearest Neighbor (_connect_neighbors):

Once the closest node is identified, the new node is connected to this node. This connection is mutual, meaning the new node also becomes a neighbor to the closest node.

7. Prune Excess Neighbors (_prune_neighbors):

If the number of neighbors exceeds a certain threshold (max_neighbours), the algorithm prunes the neighbors to keep only the closest ones. This ensures the graph remains manageable and efficient for searching.

8. Add Node to All Appropriate Layers (add_node):

Finally, the new node is added to the layers from the bottom (level 0) up to its assigned level. This ensures that the node is accessible from all levels below its assigned level.

In [116]:
# Add numpy seed for reproducibility
np.random.seed(32)

class HNSW:
    def __init__(self, max_level, max_neighbours=200, level_probability=0.5):
        self.max_level = max_level
        self.max_neighbours = max_neighbours
        self.level_probability = level_probability
        self.layers = {i: [] for i in range(max_level + 1)}
        self.entry_point = None

    # ------------------------ #
    #    Adding Node
    # ------------------------ #
    def add_node(self, embedding, label=None):
        level = self._get_random_level()
        new_node = HNSWNode(embedding, self.max_level, label=label)
        if self.entry_point is None:
            self.entry_point = new_node
        else:
            self._insert_node(new_node, level)
        for l in range(level + 1):
            self.layers[l].append(new_node)

    def _get_random_level(self):
        level = 0
        while np.random.rand() < self.level_probability and level < self.max_level:
            level += 1
        return level

    # ------------------------------------------------ #
    #    Adding Node when Entrypoint is not Null
    # ------------------------------------------------ #
    def _insert_node(self, new_node, level):
        current_node = self.entry_point
        # list: [level_new_node, level_new_node-1, ..., 0]
        for l in reversed(range(level + 1)):
            current_node = self._search_layer(new_node.embedding, current_node, l)
            self._connect_neighbors(new_node, current_node, l)

    def _search_layer(self, query, entry_point, layer):
        current_node = entry_point
        while True:
            neighbors = current_node.neighbors[layer]
            
            # Return the closest HNSWNode in that layer otherwise None
            closest = self._find_closest(query, neighbors)
            
            # if in that level there is no neighbor, then break
            # if the distance of the query to the closest neighbor is greater than the distance of the query to the current node, then break
            if closest is None or self._distance(query, closest.embedding) >= self._distance(query, current_node.embedding):
                # If True (Neighbor Not Closer) then break
                break
            current_node = closest
        return current_node

    def _connect_neighbors(self, new_node, closest_node, level):
        if new_node.label != closest_node.label:
            closest_node.add_neighbor(new_node, level)
            new_node.add_neighbor(closest_node, level)
        if len(new_node.neighbors[level]) > self.max_neighbours:
            new_node.neighbors[level] = self._prune_neighbors(new_node.neighbors[level], new_node.embedding)

    def _prune_neighbors(self, neighbors, embedding):
        distances = self._distance_matrix(embedding, [neighbor.embedding for neighbor in neighbors])
        pruned_indices = np.argsort(distances)[:self.max_neighbours]
        return [neighbors[i] for i in pruned_indices]

    def _find_closest(self, query, neighbors):
        if not neighbors:
            return None
        neighbor_embeddings = np.array([neighbor.embedding for neighbor in neighbors])
        distances = self._distance_matrix(query, neighbor_embeddings)
        closest_idx = np.argmin(distances)
        return neighbors[closest_idx]

    def _distance_matrix(self, vec, matrix):
        # Calculate Euclidean distance in a vectorized manner
        return np.linalg.norm(matrix - vec, axis=1)

    def _distance(self, vec1, vec2):
        return np.linalg.norm(vec1 - vec2)
    
    # ------------------------------------------------ #
    #    Inference
    # ------------------------------------------------ #
    def search_knn(self, query_embedding, k):
        # Start the search from the entry point at the top level
        current_node = self.entry_point
        for level in reversed(range(self.max_level + 1)):
            current_node = self._search_layer(query_embedding, current_node, level)
        
        # Now perform a search in the bottom layer to find the top K nearest neighbors
        neighbors = current_node.neighbors[0]  # Start with neighbors in the bottom level (level 0)
        candidates = [current_node] + neighbors  # Include the current node itself
        
        # Calculate distances from the query to all candidates
        distances = [self._distance(query_embedding, node.embedding) for node in candidates]
        
        # Sort candidates by distance
        sorted_indices = np.argsort(distances)
        
        # Select the top K closest nodes
        top_k_indices = sorted_indices[:k]
        
        # Return the top K closest nodes (or embeddings)
        top_k_nodes = [candidates[i] for i in top_k_indices]
        
        return top_k_nodes

    
    
class HNSW_V2:
    """
    V2: we use multiple entry points to search in parallel in a given layer
    """
    def __init__(self, max_level, max_neighbours=200, level_probability=0.5):
        self.max_level = max_level
        self.max_neighbours = max_neighbours
        self.level_probability = level_probability
        self.layers = {i: [] for i in range(max_level + 1)}
        self.entry_point = None

    def add_node(self, embedding, label=None):
        level = self._get_random_level()
        new_node = HNSWNode(embedding, self.max_level, label=label)
        if self.entry_point is None:
            self.entry_point = new_node  # Set entry_point only when it's None
        self._insert_node(new_node, level)
        for l in range(level + 1):
            self.layers[l].append(new_node)


    def _get_random_level(self):
        level = 0
        while np.random.rand() < self.level_probability and level < self.max_level:
            level += 1
        return level

    def _insert_node(self, new_node, level):
        current_node = self.entry_point
        for l in reversed(range(level + 1)):
            current_node = self._search_layer(new_node.embedding, current_node, l)
            self._connect_neighbors(new_node, current_node, l)

    def _search_layer(self, query, entry_point, layer):
        # Select 3 entry points with the most neighbors
        entry_points = self._select_top_entry_points(layer)
        
        # Perform searches from each entry point
        candidates = []
        for ep in entry_points:
            result = self._single_search_layer(query, ep, layer)
            if result is not None:
                candidates.append(result)

        # Check if candidates list is empty
        if not candidates:
            return entry_point  # or return None, depending on the desired behavior
        
        # Find the closest node among all candidates
        closest = min(candidates, key=lambda node: self._distance(query, node.embedding))
        return closest


    def _select_top_entry_points(self, layer):
        # Sort nodes in the layer by the number of neighbors, in descending order
        sorted_nodes = sorted(self.layers[layer], key=lambda node: len(node.neighbors[layer]), reverse=True)
        
        # Select the top 3 nodes with the most neighbors
        top_entry_points = sorted_nodes[:3]
        
        return top_entry_points

    def _single_search_layer(self, query, entry_point, layer):
        current_node = entry_point
        while True:
            neighbors = current_node.neighbors[layer]
            closest = self._find_closest(query, neighbors)
            if closest is None or self._distance(query, closest.embedding) >= self._distance(query, current_node.embedding):
                break
            current_node = closest
        return current_node

    def _connect_neighbors(self, new_node, closest_node, level):
        if new_node.label != closest_node.label:
            closest_node.add_neighbor(new_node, level)
            new_node.add_neighbor(closest_node, level)
        if len(new_node.neighbors[level]) > self.max_neighbours:
            new_node.neighbors[level] = self._prune_neighbors(new_node.neighbors[level], new_node.embedding)

    def _prune_neighbors(self, neighbors, embedding):
        distances = self._distance_matrix(embedding, [neighbor.embedding for neighbor in neighbors])
        pruned_indices = np.argsort(distances)[:self.max_neighbours]
        return [neighbors[i] for i in pruned_indices]

    def _find_closest(self, query, neighbors):
        if not neighbors:
            return None
        neighbor_embeddings = np.array([neighbor.embedding for neighbor in neighbors])
        distances = self._distance_matrix(query, neighbor_embeddings)
        closest_idx = np.argmin(distances)
        return neighbors[closest_idx]

    def _distance_matrix(self, vec, matrix):
        return np.linalg.norm(matrix - vec, axis=1)

    def _distance(self, vec1, vec2):
        return np.linalg.norm(vec1 - vec2)
    
    def search_knn(self, query_embedding, k):
        # Start the search from the top level using multiple entry points
        current_candidates = []
        
        for level in reversed(range(self.max_level + 1)):
            # Get the top 3 entry points with the most neighbors at the current level
            entry_points = self._select_top_entry_points(level)
            
            # Perform searches from each entry point and collect candidates
            level_candidates = []
            for entry_point in entry_points:
                candidate = self._single_search_layer(query_embedding, entry_point, level)
                if candidate:
                    level_candidates.append(candidate)
            
            # Merge level candidates into the current candidates
            current_candidates.extend(level_candidates)
        
        # Remove duplicates from candidates
        current_candidates = list(set(current_candidates))
        
        # Perform a final selection of the top K nearest neighbors from the bottom level
        if current_candidates:
            distances = [self._distance(query_embedding, node.embedding) for node in current_candidates]
            sorted_indices = np.argsort(distances)
            top_k_nodes = [current_candidates[i] for i in sorted_indices[:k]]
        else:
            top_k_nodes = []

        return top_k_nodes


## Let's Debug V1

In [105]:
%%time
np.random.seed(0)
hnsw = HNSW(max_level=3)
for i, emb in enumerate(embeddings):
    hnsw.add_node(emb, label=f"emb{i}")

CPU times: user 128 ms, sys: 10.4 ms, total: 138 ms
Wall time: 164 ms


### Create a fake embedding

In [75]:
embf = 0.5*embeddings[0] + 0.5*embeddings[1]

In [76]:
np.random.seed(1)
hnsw.add_node(embf, label="embf")

## Let's Debug V2

In [112]:
%%time
np.random.seed(0)
hnsw2 = HNSW_V2(max_level=3)
for i, emb in enumerate(embeddings):
    hnsw2.add_node(emb, label=f"emb{i}")

CPU times: user 847 ms, sys: 19.8 ms, total: 867 ms
Wall time: 904 ms


### Create a fake embedding

In [68]:
embf = 0.5*embeddings[0] + 0.5*embeddings[1]

In [69]:
np.random.seed(1)
hnsw2.add_node(embf, label="embf")

# Step 3: Finding K-Nearest Neighbors

In [85]:
# Add numpy seed for reproducibility
np.random.seed(63)

# Example: Create 1000 random embeddings with 128 dimensions
num_embeddings = 500
dim = 128
test_embeddings = generate_random_embeddings(num_embeddings, dim)
_ = sizeof(test_embeddings)

Size: 0.244 MB


# Benchmark with KNN

To change:
1. Cosine Similarity

In [117]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import recall_score, precision_score, label_ranking_average_precision_score
import time

In [118]:
from scipy.stats import rankdata
import numpy as np

def mean_reciprocal_rank(y_true, y_pred):
    mrrs = []
    for true_labels, pred_scores in zip(y_true, y_pred):
        # Rank predictions in descending order
        ranks = rankdata(-pred_scores, method='ordinal')
        # Get the rank of the true label
        true_rank = ranks[np.argmax(true_labels)]
        # Calculate reciprocal rank
        mrrs.append(1.0 / true_rank)
    return np.mean(mrrs)


# Example usage
y_true = np.array([[0, 0, 1, 0, 0], [0, 0, 0, 1, 0]])
y_pred = np.array([[0.1, 0.2, 0.9, 0.4, 0.3], [0.05, 0.1, 0.6, 0.8, 0.4]])
mrr = mean_reciprocal_rank(y_true, y_pred)
print(f"MRR: {mrr}")

MRR: 1.0


## 2. Exact KNN with Scikit-learn

In [119]:
# Define parameters
K = 10  # Set the value of K
dim = 128

# Initialize the NearestNeighbors model from scikit-learn
knn_model = NearestNeighbors(n_neighbors=K, algorithm='brute', metric='euclidean')

# Fit the model to the test embeddings
knn_model.fit(embeddings)

# Perform exact KNN search
start_time_knn = time.time()
distances, indices = knn_model.kneighbors(test_embeddings)
knn_exact_time = time.time() - start_time_knn

# Store the exact KNN results as the real ranking of node labels
exact_knn_labels = [indices[i] for i in range(len(test_embeddings))]
exact_knn_distances = [distances[i] for i in range(len(test_embeddings))]


## 3. HNSW KNN Search

In [120]:
def benchmark_hnsw(hnsw_class, embeddings, test_embeddings, K):
    """
    Perform KNN search for each embedding using the given HNSW class (HNSW or HNSW_V2).
    """
    np.random.seed(0)
    # Initialize the HNSW model
    hnsw = hnsw_class(max_level=3)

    # Adding nodes to HNSW
    start_time_hnsw_add = time.time()
    for i, emb in enumerate(embeddings):
        hnsw.add_node(emb, label=f"emb{i}")
    hnsw_add_time = time.time() - start_time_hnsw_add

    # Perform HNSW search_knn for each test embedding
    hnsw_knn_labels = []
    start_time_hnsw_search = time.time()
    for emb in test_embeddings:
        top_k_nodes = hnsw.search_knn(emb, k=K)
        hnsw_knn_labels.append([int(node.label.replace('emb', '')) for node in top_k_nodes])
    hnsw_search_time = time.time() - start_time_hnsw_search

    return hnsw_knn_labels, hnsw_add_time, hnsw_search_time


In [121]:
# Benchmark HNSW (V1)
hnsw_v1_labels, hnsw_v1_add_time, hnsw_v1_search_time = benchmark_hnsw(HNSW, embeddings, test_embeddings, K)

# Benchmark HNSW_V2
hnsw_v2_labels, hnsw_v2_add_time, hnsw_v2_search_time = benchmark_hnsw(HNSW_V2, embeddings, test_embeddings, K)


## 4. Ranking Metrics: Recall@K, Precision@K, MRR

In [107]:
# Helper functions for ranking metrics
def recall_at_k(actual, predicted, k):
    actual_set = set(actual[:k])
    predicted_set = set(predicted[:k])
    return len(actual_set & predicted_set) / len(actual_set)

def precision_at_k(actual, predicted, k):
    actual_set = set(actual[:k])
    predicted_set = set(predicted[:k])
    return len(actual_set & predicted_set) / k

def mean_reciprocal_rank(actual, predicted):
    for i, p in enumerate(predicted):
        if p in actual:
            return 1.0 / (i + 1)
    return 0.0

In [122]:
# Initialize metrics
def calculate_metrics(exact_labels, hnsw_labels, K):
    recall_scores = []
    precision_scores = []
    mrr_scores = []

    for i in range(num_embeddings):
        recall = recall_at_k(exact_labels[i], hnsw_labels[i], K)
        precision = precision_at_k(exact_labels[i], hnsw_labels[i], K)
        mrr = mean_reciprocal_rank(exact_labels[i], hnsw_labels[i])

        recall_scores.append(recall)
        precision_scores.append(precision)
        mrr_scores.append(mrr)

    average_recall = np.mean(recall_scores)
    average_precision = np.mean(precision_scores)
    average_mrr = np.mean(mrr_scores)

    return average_recall, average_precision, average_mrr

## 5. Create Pandas DataFrame for Results

In [123]:
# Calculate metrics for HNSW (V1)
v1_recall, v1_precision, v1_mrr = calculate_metrics(exact_knn_labels, hnsw_v1_labels, K)

# Calculate metrics for HNSW_V2
v2_recall, v2_precision, v2_mrr = calculate_metrics(exact_knn_labels, hnsw_v2_labels, K)



# Create a pandas DataFrame to store the results
benchmark_df = pd.DataFrame({
    'method': ['HNSW_V1', 'HNSW_V2'],
    'num_embeddings': [num_embeddings, num_embeddings],
    'embedding_dim': [dim, dim],
    'average_recall': [v1_recall, v2_recall],
    'average_precision': [v1_precision, v2_precision],
    'average_mrr': [v1_mrr, v2_mrr],
    'add_time': [hnsw_v1_add_time, hnsw_v2_add_time],
    'search_time': [hnsw_v1_search_time, hnsw_v2_search_time],
    'knn_exact_search_time': [knn_exact_time, knn_exact_time]
})

In [124]:
benchmark_df

,method,num_embeddings,embedding_dim,average_recall,average_precision,average_mrr,add_time,search_time,knn_exact_search_time
0,HNSW_V1,500,128,0.0396,0.0396,1.0,0.122691,0.110658,0.007073
1,HNSW_V2,500,128,0.2720,0.2720,1.0,0.958108,1.296570,0.007073
